In [1]:
! pip install -q kaggle

In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"saiyedshizain","key":"b2c3435e51268984e68eaff351f9fa24"}'}

In [6]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets list

ref                                                                  title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------------  ------------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
jayaantanaath/student-habits-vs-academic-performance                 Student Habits vs Academic Performance                  19512  2025-04-12 10:49:08.663000           3529         57  1.0              
adilshamim8/student-depression-dataset                               Student Depression Dataset                             467020  2025-03-13 03:12:30.423000          20830        334  1.0              
zahidmughal2343/supplement-sales-data                                Supplement Sales Data                                   66800  2025-04-11 15:01:00.227000           1367         28

In [9]:
!kaggle datasets download -d oddrationale/mnist-in-csv
!unzip mnist-in-csv.zip

Dataset URL: https://www.kaggle.com/datasets/oddrationale/mnist-in-csv
License(s): CC0-1.0
Archive:  mnist-in-csv.zip
  inflating: mnist_test.csv          
  inflating: mnist_train.csv         


In [10]:
!ls

kaggle.json  mnist-in-csv.zip  mnist_test.csv  mnist_train.csv	sample_data


In [11]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [12]:
train_data = pd.read_csv('mnist_train.csv')
test_data = pd.read_csv('mnist_test.csv')

In [13]:
data = np.array(train_data)
m, n = data.shape
np.random.shuffle(data)

In [14]:
# Creating development (validation) and training sets
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255   # Normalizing pixel values

In [15]:
# Creating Training Dataset from remaining
data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255   # Normalizing pixel values
_,m_train = X_train.shape

In [16]:
Y_train

array([1, 0, 0, ..., 0, 5, 2])

In [22]:
# Define Required Mathematical Functions
def init_params():
  # Subtract 0.5 from each para. to keep the range b/w -0.5 to +0.5
  W1 = np.random.rand(10, 784) - 0.5
  b1 = np.random.rand(10, 1) - 0.5
  W2 = np.random.rand(10, 10) - 0.5
  b2 = np.random.rand(10, 1) - 0.5
  return W1, b1, W2, b2

def ReLU(Z):
  return np.maximum(Z, 0)

def softmax(Z):
  A = np.exp(Z) / sum(np.exp(Z))

def forward_propogation(W1, b1, W2, b2, X):     # X is A0
  Z1 = W1.dot(X) + b1
  A1 = ReLU(Z1)
  Z2 = W2.dot(A1) + b2
  A2 = softmax(Z2)
  return Z1, A1, Z2, A2

def one_hot_encode(Y):
  one_hot = np.zeros((Y.size, Y.max() + 1))
  one_hot[np.arange(Y.size), Y] = 1
  one_hot = one_hot.T
  return one_hot

def derivative_ReLU(Z):
  return Z > 0

def back_propogation(Z1, A1, Z2, A2, W2, X, Y):
  m = Y.size
  one_hot_Y = one_hot_encode(Y)
  dZ2 = A2 - one_hot_Y
  dW2 = 1 / m * dZ2.dot(A1.T)
  db2 = 1 / m * np.sum(dZ2, 2)
  dZ1 = W2.T.dot(dZ2) * derivative_ReLU(Z1)
  dW1 = 1 / m * dZ1.dot(X.T)
  db1 = 1 / m * np.sum(dZ1, 2)
  return dW1, db1, dW2, db2

def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
  W1 = W1 - alpha * dW1
  b1 = b1 - alpha * db1
  W2 = W2 - alpha * dW2
  b2 = b2 - alpha * db2
  return W1, b1, W2, b2

In [23]:
# Define Gradient Descent & it's required functions
def get_predictions(A2):
  return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
  print(predictions, Y)
  return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
  W1, b1, W2, b2 = init_params()
  for i in range(iterations):
    Z1, A1, Z2, A2 = forward_propogation(W1, b1, W2, b2, X)
    dW1, db1, dW2, db2 = back_propogation(Z1, A1, Z2, A2, W2, X, Y)
    W1, b2, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
    if i % 50 == 0:
      print("Iteration:", i)
      print("Accuracy:", get_accuracy(get_predictions(A2), Y))
    return W1, b1, W2, b2

In [24]:
# Train the model
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 500, 0.1)

TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'